### Insert Your Imports In This Cell

In [2]:
import time
from typing import Tuple, List
import heapq

### Insert Your Implementations In This Cell

In [47]:
def replacer(s, newstring, index, nofail=False):
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")
    if index < 0:  
        return newstring + s
    if index > len(s):
        return s + newstring
    return s[:index] + newstring + s[index + 1:]

def isOver(cur: List[str]) -> bool:
    for r in range(0, len(cur)):
        if (cur[r].count("1") > 0 or cur[r].count("2") > 0 or cur[r].count("3") > 0):
            return False
    return True

def removeP(cur: List[str]) -> bool:
    for r in range(0, len(cur)):
        if (cur[r].count("1") > 0 or cur[r].count("3") > 0):
            return False
    return True

def removeQ(cur: List[str]) -> bool:
    for r in range(0, len(cur)):
        if (cur[r].count("2") > 0 or cur[r].count("3") > 0):
            return False
    return True

def isCellValid(cur: List[str], agent: int, row: int, col: int) -> bool:
    if (row < 0 or row >= len(cur) or col < 0 or col >= len(cur[-1])):
        return False
    if (cur[row][col] == "%"):
        return False
    if (agent == 1 and (cur[row][col] == "2" or cur[row][col] == "Q")):
        return False
    if (agent == 2 and (cur[row][col] == "1" or cur[row][col] == "P")):
        return False
    return True

def getSuccessor(cur: List[str]) -> List[Tuple[List[str], str, int, int]]:
    successors = list()
    for r in range(0, len(cur)):
        if (cur[r].count("Q") > 0):
            c = cur[r].index("Q")
            #left
            if (isCellValid(cur, 2, r, c-1)):
                left = cur[:]
                left[r] = replacer(left[r], ".", c)
                left[r] = replacer(left[r], "Q", c-1)
                eaten = 0
                if ((cur[r][c-1] == "2" or cur[r][c-1] == "3")):
                    eaten = 1
                    if (removeQ(left)):
                        left[r] = replacer(left[r], ".", c-1)
                
                successors.append((left, "L", eaten, 2))
            #down
            if (isCellValid(cur, 2, r+1, c)):
                down = cur[:]
                down[r] = replacer(down[r], ".", c)
                down[r+1] = replacer(down[r+1], "Q", c)
                eaten = 0
                if ((cur[r+1][c] == "2" or cur[r+1][c] == "3")):
                    eaten = 1
                    if (removeQ(down)):
                        down[r+1] = replacer(down[r+1], ".", c)
                
                successors.append((down, "D", eaten, 2))
            #right
            if (isCellValid(cur, 2, r, c+1)):
                right = cur[:]
                right[r] = replacer(right[r], ".", c)
                right[r] = replacer(right[r], "Q", c+1)
                eaten = 0
                if ((cur[r][c+1] == "2" or cur[r][c+1] == "3")):
                    eaten = 1
                    if (removeQ(right)):
                        right[r] = replacer(right[r], ".", c+1)
                successors.append((right, "R", eaten, 2))
            #up
            if (isCellValid(cur, 2, r-1, c)):
                up = cur[:]
                up[r] = replacer(up[r], ".", c)
                up[r-1] = replacer(up[r-1], "Q", c)
                eaten = 0
                if ((cur[r-1][c] == "2" or cur[r-1][c] == "3")):
                    eaten = 1
                    if (removeQ(up)):
                        up[r-1] = replacer(up[r-1], ".", c)
                successors.append((up, "U", eaten, 2))
    for r in range(0, len(cur)):
        if (cur[r].count("P") > 0):
            c = cur[r].index("P")
            #left
            if (isCellValid(cur, 1, r, c-1)):
                left = cur[:]
                left[r] = replacer(left[r], ".", c)
                left[r] = replacer(left[r], "P", c-1)
                eaten = 0
                if ((cur[r][c-1] == "1" or cur[r][c-1] == "3")):
                    eaten = 1
                    if (removeP(left)):
                        left[r] = replacer(left[r], ".", c-1)
                successors.append((left, "L", eaten, 1))
            #down
            if (isCellValid(cur, 1, r+1, c)):
                down = cur[:]
                down[r] = replacer(down[r], ".", c)
                down[r+1] = replacer(down[r+1], "P", c)
                eaten = 0
                if ((cur[r+1][c] == "1" or cur[r+1][c] == "3")):
                    eaten = 1
                    if (removeP(down)):
                        down[r+1] = replacer(down[r+1], ".", c)
                successors.append((down, "D", eaten, 1))
            #right
            if (isCellValid(cur, 1, r, c+1)):
                right = cur[:]
                right[r] = replacer(right[r], ".", c)
                right[r] = replacer(right[r], "P", c+1)
                eaten = 0
                if ((cur[r][c+1] == "1" or cur[r][c+1] == "3")):
                    eaten = 1
                    if (removeP(right)):
                        right[r] = replacer(right[r], ".", c+1)
                successors.append((right, "R", eaten, 1))
            #up
            if (isCellValid(cur, 1, r-1, c)):
                up = cur[:]                        
                up[r] = replacer(up[r], ".", c)
                up[r-1] = replacer(up[r-1], "P", c)
                eaten = 0
                if ((cur[r-1][c] == "1" or cur[r-1][c] == "3")):
                    eaten = 1
                    if (removeP(up)):
                        up[r-1] = replacer(up[r-1], ".", c)
                successors.append((up, "U", eaten, 1))
        
                
   #no action
    # if (len(successors) == 0):
    #     successors.append((cur[:], "-", 0))
    return successors

def dfs(map: List[str], limit: int, explored: List):
    # frontier = [(map, 0, "", "")]
    def checkLimit(cur):
        if (cur[1] == limit - 1 or cur[1] == limit - 2):
            return True
        return False
    frontier = list(filter(checkLimit, explored)) if len(explored) > 0 else [(map, 0, "", "")]
    frontier.reverse()
    explored.clear()
    while(frontier):
        cur = frontier.pop()
        curMap = cur[0]
        hasExplored = False
        for i in range(0, len(explored)):
            n = explored[i]
            if (n[0] == curMap and n[1] == cur[1]):
                curTotalMoves = len(cur[2]) + len(cur[3])
                exploredTotalMoves = len(n[2]) + len(n[3])
                if (curTotalMoves >= exploredTotalMoves):
                    hasExplored = True
                else:
                    explored.pop(i)
                break

        if (not hasExplored):
            explored.append(cur)
            if (isOver(curMap)):
                return cur[2], cur[3]
            if (cur[1] + 1 < limit):
                successors = getSuccessor(curMap)
                for successor in successors:
                    agent = successor[3]
                    if (agent == 1):
                        moves = cur[2] + successor[1]
                        frontier.append((successor[0], cur[1] + 1, moves, cur[3]))
                    else:
                        moves = cur[3] + successor[1]
                        frontier.append((successor[0], cur[1] + 1, cur[2], moves))
    return -1


def IDS(board_map: List[str]) -> Tuple[str, str]:
    p_steps, q_steps = list(), list()
    explored = [(board_map, 0, "", "")]
    depth = 1
    while True:
        result = dfs(board_map, depth, explored)
        if (result != -1):
            return result
        depth += 1
    # return "".join(p_steps), "".join(q_steps)

def getHeuristic(cur: List[str], eatenP, eatenQ) -> int:
    edibleP, edibleQ, edibleBoth = list(), list(), list()
    P = (-1,- 1)
    Q = (-1, -1)
    for r in range(0, len(cur)):
        for c in range(0, len(cur[r])):
            if (cur[r][c] == "P"):
                P = (r,c)
            if (cur[r][c] == "Q"):
                Q = (r,c)
            if (cur[r][c] == "1"):
                edibleP.append((r,c))
            if (cur[r][c] == "2"):
                edibleQ.append((r,c))
            if (cur[r][c] == "3"):
                edibleBoth.append((r,c))
    lowestP = 0
    lowestQ = 0
    curLowestP = sys.maxsize
    curLowestQ = sys.maxsize
    if (P != (-1,-1) and eatenP == 0):
        for e in edibleP:
            distance = abs(P[0] - e[0]) + abs(P[1] - e[1])
            if (distance < curLowestP):
                curLowestP = distance
        lowestP = curLowestP
    if (Q != (-1, -1) and eatenQ == 0):
        for e in edibleQ:
            distance = abs(Q[0] - e[0]) + abs(Q[1] - e[1])
            if (distance < curLowestQ):
                curLowestQ = distance
        
        lowestQ = curLowestQ
    for i in range(0, len(edibleBoth)):
        distanceP = abs(P[0] - edibleBoth[i][0]) + abs(P[1] - edibleBoth[i][1]) if P != (-1, -1) and eatenP == 0 else 99999
        distanceQ = abs(Q[0] - edibleBoth[i][0]) + abs(Q[1] - edibleBoth[i][1]) if Q != (-1, -1) and eatenQ == 0 else 99999
        if (distanceP == distanceQ):
            if (distanceP < curLowestP and distanceQ < curLowestQ):
                curLowestP = distanceP
                
            elif (distanceP < curLowestP):
                curLowestP = distanceP
            elif (distanceQ < curLowestQ):
                curLowestQ = distanceQ

        if (distanceP < distanceQ and distanceP < curLowestP):
            curLowestP = distanceP
        if (distanceQ < distanceP and distanceQ < curLowestQ):
            curLowestQ = distanceQ

    if (P != (-1, -1) and eatenP == 0):
        lowestP = curLowestP
    if (Q != (-1, -1) and eatenQ == 0):
        lowestQ = curLowestQ
    return lowestQ + lowestP

def getCost(pathP, pathQ):
    return len(pathP) + len(pathQ)


def Astar(board_map: List[str]) -> Tuple[str, str]:
    p_steps, q_steps = list(), list()
    h0 = getHeuristic(board_map, 0, 0)
    frontier = [(h0,(board_map, 0, "", ""))]
    heapq.heapify(frontier)
    explored = []
    count = 0
    while (frontier):
        cur = heapq.heappop(frontier)[1]        
        curMap = cur[0]
        hasExplored = False
        for i in range(0, len(explored)):
            n = explored[i]
            if (n[0] == curMap):
                curTotalMoves = len(cur[2]) + len(cur[3])
                exploredTotalMoves = len(n[2]) + len(n[3])
                if (curTotalMoves >= exploredTotalMoves):
                    hasExplored = True
                else:
                    explored.pop(i)
                break
        if (not hasExplored):
            explored.append(cur)
            if (isOver(curMap)):
                return cur[2], cur[3]
            successors = getSuccessor(curMap)
            for successor in successors:
                agent = successor[3]
                if (agent == 1):
                    moves = cur[2] + successor[1]
                    fValue = getCost(moves, cur[3]) + getHeuristic(successor[0], successor[2], 0)
                    heapq.heappush(frontier, (fValue, (successor[0], cur[1] + 1, moves, cur[3])))
                else:
                    moves = cur[3] + successor[1]
                    fValue = getCost(cur[2], moves) + getHeuristic(successor[0], 0, successor[2])
                    heapq.heappush(frontier, (fValue, (successor[0], cur[1] + 1, cur[2], moves)))
        count += 1
    return -1


### Utility Functions

In [20]:
def read_file(fileaddr):
    with open(fileaddr) as file:
        table = file.readlines()
        
    return table

def print_map(board_map):
    print(f"----------- Map -----------")
    for line in board_map:
        print(line, end="")
    print("\n---------------------------")

def print_agent_steps(steps, agent_name):
    print(f"Agent {agent_name}: Num of Steps: {len(steps)} - Steps: {steps}") # steps should be a string of U, R, D, L characters for Up, Right, Down, and Left movements, respectively. An example would be 'URRURRUL'

def print_steps_time(p_steps, q_steps, elpased_time, algorithm_name):
    print(f"----------- {algorithm_name} -----------")
    print_agent_steps(p_steps, "P")
    print_agent_steps(q_steps, "Q")
    print(f"Elapsed Time: {elpased_time} Seconds")
    print("----------------------------")

### random test

In [35]:
if __name__=="__main__":
    test_case_number = 2 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")

    b = ['%%%%%%\n', '%...1%\n', '%....%\n', '%...P%\n', '%%.%%%\n', '%...3%\n', '%1...%\n', '%%%.%%\n', '%3.Q.%\n', '%%%%%%']
    print(getHeuristic(b, 0, 0))


    

4


In [46]:
if __name__=="__main__":
    test_case_number = 1 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")

    # b = ['%%%%%%\n', '%...P%\n', '%....%\n', '%....%\n', '%%.%%%\n', '%...3%\n', '%1...%\n', '%%%.%%\n', '%3.Q.%\n', '%%%%%%']
    # print(getHeuristic(b, 1, 0))


    # Running IDS
    # tick = time.time()
    # p_steps, q_steps = IDS(board_map)
    # tock = time.time()
    # print_steps_time(p_steps, q_steps, tock-tick, "IDS")


    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

KeyboardInterrupt: 

### Test 1

In [ ]:
if __name__=="__main__":
    test_case_number = 1 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # # Running A* Search
    # tick = time.time()c
    # p_steps, q_steps = Astar(board_map)
    # tock = time.time()
    # print_steps_time(p_steps, q_steps, tock-tick, "A*")

### Test 2

In [48]:
if __name__=="__main__":
    test_case_number = 2 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")
    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- IDS -----------
Agent P: Num of Steps: 14 - Steps: UUDDLLDDRRDLLL
Agent Q: Num of Steps: 3 - Steps: LLL
Elapsed Time: 8.364536046981812 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 14 - Steps: UUDDLLDDRRDLLL
Agent Q: Num of Steps: 3 - Steps: LLL
Elapsed Time: 0.43523693084716797 Seconds
----------------------------


### Test 3

In [49]:
if __name__=="__main__":
    test_case_number = 3 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- IDS -----------
Agent P: Num of Steps: 10 - Steps: RDLLLLLLLL
Agent Q: Num of Steps: 10 - Steps: LRRRRRRRRR
Elapsed Time: 6.228408098220825 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 10 - Steps: RDLLLLLLLL
Agent Q: Num of Steps: 10 - Steps: LRRRRRRRRR
Elapsed Time: 0.45452427864074707 Seconds
----------------------------


### Test 4

In [50]:
if __name__=="__main__":
    test_case_number = 4 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- IDS -----------
Agent P: Num of Steps: 5 - Steps: RRDDD
Agent Q: Num of Steps: 12 - Steps: DLLLLLLULULL
Elapsed Time: 111.68963623046875 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 5 - Steps: RRDDD
Agent Q: Num of Steps: 12 - Steps: DLLLLLLLUULL
Elapsed Time: 0.5625882148742676 Seconds
----------------------------


### Test 5

In [51]:
if __name__=="__main__":
    test_case_number = 5 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")

    # # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- IDS -----------
Agent P: Num of Steps: 8 - Steps: LLDDDRLL
Agent Q: Num of Steps: 5 - Steps: LUUUR
Elapsed Time: 0.13284897804260254 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 8 - Steps: LLDDDRLL
Agent Q: Num of Steps: 5 - Steps: LUUUR
Elapsed Time: 0.03226590156555176 Seconds
----------------------------
